In [4]:
print('CODING ATTENTION MECHANISM')

CODING ATTENTION MECHANISM


In [5]:
import torch
inputs=torch.tensor(torch.rand(6,4))
inputs

/var/folders/hp/y1d826dx5fb1405ppnjjm4700000gn/T/ipykernel_4906/935272016.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs=torch.tensor(torch.rand(6,4))


tensor([[0.8398, 0.8042, 0.1213, 0.5309],
        [0.6646, 0.4077, 0.0888, 0.2429],
        [0.7053, 0.6216, 0.9188, 0.0185],
        [0.8741, 0.0560, 0.9659, 0.0073],
        [0.3628, 0.4197, 0.6444, 0.0099],
        [0.5925, 0.9631, 0.6958, 0.9157]])

In [6]:
# finding z3 a context vector for fourth token

query4=inputs[3]
query4 @ inputs.T  # 1x4  @   6x4 ------>  1x4 @ 4x6 = 1x6  

tensor([0.9002, 0.6913, 1.5390, 1.7004, 0.9632, 1.2506])

In [7]:
#majddori method 


query4=inputs[3]
attention_score_4=torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attention_score_4[i]=torch.dot(x_i,query4)
print(attention_score_4,attention_score_4.shape)    
    



tensor([0.9002, 0.6913, 1.5390, 1.7004, 0.9632, 1.2506]) torch.Size([6])


In [8]:
for i in inputs:
    print(i)

tensor([0.8398, 0.8042, 0.1213, 0.5309])
tensor([0.6646, 0.4077, 0.0888, 0.2429])
tensor([0.7053, 0.6216, 0.9188, 0.0185])
tensor([0.8741, 0.0560, 0.9659, 0.0073])
tensor([0.3628, 0.4197, 0.6444, 0.0099])
tensor([0.5925, 0.9631, 0.6958, 0.9157])


In [9]:
majduri_dot_product=0
for idx,element in enumerate(inputs[0]):
    majduri_dot_product+=inputs[0][idx] * query4[idx]
print(majduri_dot_product)
print(torch.dot(inputs[0],query4))



tensor(0.9002)
tensor(0.9002)


In [10]:
#normalizing 

attn_weights_4_normlaized = attention_score_4 / attention_score_4.sum()
print(f'Attention scores without normalization : {attention_score_4}')
print(f'Attention score with normalization     : {attn_weights_4_normlaized}')

Attention scores without normalization : tensor([0.9002, 0.6913, 1.5390, 1.7004, 0.9632, 1.2506])
Attention score with normalization     : tensor([0.1278, 0.0981, 0.2185, 0.2414, 0.1367, 0.1775])


In [11]:
attn_weights_4_normlaized.sum()

tensor(1.)

In [12]:
# applying softmax instead of normalization

def naive_softmax(x): 
    print(torch.exp(x).shape) 
    return torch.exp(x)/torch.exp(x).sum(dim=0)

attn_weights_4_sft=naive_softmax(attention_score_4)

attn_weights_4_sft

torch.Size([6])


tensor([0.1188, 0.0964, 0.2251, 0.2645, 0.1265, 0.1687])

In [13]:
attn_weights_4_sft.sum()

tensor(1.0000)

In [14]:
# optimized way by pytorch

attn_weights_4_sft_optzd=torch.softmax(attention_score_4,dim=0)

print(f'Attention weights using softmax by pytorch {attn_weights_4_sft_optzd}')

print(f'Attnetion scores sum is {attn_weights_4_sft_optzd.sum()}')

Attention weights using softmax by pytorch tensor([0.1188, 0.0964, 0.2251, 0.2645, 0.1265, 0.1687])
Attnetion scores sum is 1.0


In [15]:
# okay now we will apply attention scores to their respective vectors and then sum it 

query=inputs[3] #this is the 4th token

context_vec_4=torch.zeros(query.shape)

for i,x_i in enumerate(inputs):
    context_vec_4+=attn_weights_4_sft_optzd[i] * x_i

print(context_vec_4)
    

tensor([0.6997, 0.5051, 0.6841, 0.2483])


In [16]:
torch.matmul(attn_weights_4_sft_optzd,inputs)

tensor([0.6997, 0.5051, 0.6841, 0.2483])

In [17]:
# now calculating for every token in the input 

attention_score_input=torch.empty(6,6)

for i , x_i in enumerate(inputs):
    for j , x_j in enumerate(inputs):
        attention_score_input[i,j]=torch.dot(x_i,x_j)

print(attention_score_input)        

tensor([[1.6485, 1.0258, 1.2135, 0.9002, 0.7257, 1.8426],
        [1.0258, 0.6749, 0.8083, 0.6913, 0.4719, 1.0707],
        [1.2135, 0.8083, 1.7284, 1.5390, 1.1091, 1.6728],
        [0.9002, 0.6913, 1.5390, 1.7004, 0.9632, 1.2506],
        [0.7257, 0.4719, 1.1091, 0.9632, 0.7231, 1.0766],
        [1.8426, 1.0707, 1.6728, 1.2506, 1.0766, 2.6011]])


In [18]:
torch.matmul(inputs,inputs.T)

tensor([[1.6485, 1.0258, 1.2135, 0.9002, 0.7257, 1.8426],
        [1.0258, 0.6749, 0.8083, 0.6913, 0.4719, 1.0707],
        [1.2135, 0.8083, 1.7284, 1.5390, 1.1091, 1.6728],
        [0.9002, 0.6913, 1.5390, 1.7004, 0.9632, 1.2506],
        [0.7257, 0.4719, 1.1091, 0.9632, 0.7231, 1.0766],
        [1.8426, 1.0707, 1.6728, 1.2506, 1.0766, 2.6011]])

In [19]:
attention_score_input_sft_nrmlzd=torch.softmax(attention_score_input,dim=1)

In [20]:
torch.sum(torch.softmax(attention_score_input,dim=1),dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [21]:
context_vectors=torch.matmul(attention_score_input_sft_nrmlzd,inputs)
context_vectors

tensor([[0.6864, 0.6531, 0.5439, 0.4199],
        [0.6871, 0.5944, 0.5598, 0.3479],
        [0.6851, 0.5683, 0.6572, 0.3041],
        [0.6997, 0.5051, 0.6841, 0.2483],
        [0.6786, 0.5614, 0.6342, 0.2982],
        [0.6703, 0.6960, 0.5966, 0.4804]])

In [22]:
# here we start with trainable matrices with matrix q,k,v
input_vec_token2=inputs[1]

d_in=inputs.shape[1]
d_out=3




In [23]:
import torch.nn 
torch.manual_seed(6969)

w_key=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
w_query=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
w_value=torch.nn.Parameter(torch.rand(d_in,d_out,requires_grad=False))

In [24]:
query_2=inputs[1]@w_query
key_2=inputs[1]@w_key
value_2=inputs[1]@w_value

In [25]:
print(query_2)
print(key_2)
print(value_2)


tensor([1.0028, 0.4865, 0.6946])
tensor([0.7034, 0.8019, 0.6788])
tensor([0.8855, 0.8590, 0.4732], grad_fn=<SqueezeBackward4>)


In [26]:
keys=inputs @ w_key #6x4 4x3 ---> 6x3
values=inputs @ w_value#6x4 4x3 ---> 6x3
query=inputs @ w_query #6x4 4x3 ---> 6x3

In [27]:
keys.shape

torch.Size([6, 3])

In [28]:
#now we find attention matrix we use quey vector of that token fow whihc we want to find its corresponding vector

query_2=query[1]

attention_scores2=torch.matmul(query_2,keys.T)  #1x3  @  6x3 ---> 1x6

print(attention_scores2.sum())


tensor(15.9836)


In [29]:
# now we scale the logits first then apply spftmax
attention_scores2=torch.softmax(attention_scores2 / d_out**0.5,dim=0)
ct1=torch.matmul(attention_scores2,values)  #1x6  @6x3-----> 1x3
print(attention_scores2.sum(dim=0))

tensor(1.0000)


In [30]:
ct1

tensor([1.0702, 1.4964, 1.0350], grad_fn=<SqueezeBackward4>)

In [46]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in,d_out):  
        super().__init__()
        self.w_query=nn.Parameter(torch.rand(d_in,d_out))
        self.W_key=nn.Parameter(torch.rand(d_in,d_out))
        self.W_value=nn.Parameter(torch.rand(d_in,d_out))

    def forward(self,x):
        keys= x @ self.W_key   
        print('kdone')         
        queries= x @ self.w_query
        print('qdone') 
        values=x @ self.W_value
        print('vdone') 
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5 , dim=-1
        )   
        context_vec = attn_weights @ values
        return context_vec
    

In [47]:
import torch
torch.manual_seed(69)

sa_v1=SelfAttention_v1(d_in,d_out)
print(sa_v1(inputs))

kdone
qdone
vdone
tensor([[1.1773, 1.3761, 1.1954],
        [1.0623, 1.2755, 1.0853],
        [1.1467, 1.3518, 1.1668],
        [1.0986, 1.3097, 1.1207],
        [1.0454, 1.2633, 1.0700],
        [1.2563, 1.4486, 1.2726]], grad_fn=<MmBackward0>)


In [49]:
#using nn.linear as it has he/xavier weight initializaton

class SelfAttention_v2(nn.Module):
    def __init__(self,d_in,d_out,bias_=False):
        super().__init__()

        self.w_keys_v2=nn.Linear(d_in,d_out,bias=bias_)
        self.w_query_v2=nn.Linear(d_in,d_out,bias=bias_)
        self.w_values_v2=nn.Linear(d_in,d_out,bias=bias_)

    def forward(self,x):
        keys_matrix_v2=self.w_keys_v2(x)
        values_matrix_v2=self.w_values_v2(x)
        query_matrix_v2=self.w_query_v2(x)

        attention_scores_matirx=query_matrix_v2 @ keys_matrix_v2.T

        atteniton_weights=torch.softmax(
            attention_scores_matirx / keys_matrix_v2.shape[-1]**0.5,dim=-1
        )
        context_vectors_v2=atteniton_weights @ values_matrix_v2

        return context_vectors_v2







torch.manual_seed(69)

sa_v2=SelfAttention_v2(d_in,d_out)
print(sa_v2(inputs))

tensor([[-0.2243,  0.0456, -0.0402],
        [-0.2239,  0.0622, -0.0334],
        [-0.2226,  0.0685, -0.0314],
        [-0.2230,  0.0851, -0.0246],
        [-0.2225,  0.0700, -0.0307],
        [-0.2241,  0.0335, -0.0454]], grad_fn=<MmBackward0>)


In [61]:
queries= sa_v2.w_query_v2(inputs)
keys= sa_v2.w_query_v2(inputs)
values=sa_v2.w_values_v2(inputs)

attention_scores=keys @ values.T
attention_weights=torch.softmax(
    attention_scores / keys.shape[-1] ** 0.5 ,dim=-1
)

context_vectors = attention_weights @ values


In [64]:
attention_weights.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)

In [65]:
device = torch.device("mps") 

In [66]:
device

device(type='mps')

In [81]:
mac1=torch.tensor(torch.rand(11,100)).to(device)
mac2=torch.tensor(torch.rand(11,100)).to(device)


/var/folders/hp/y1d826dx5fb1405ppnjjm4700000gn/T/ipykernel_4906/1546787270.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mac1=torch.tensor(torch.rand(11,100)).to(device)
/var/folders/hp/y1d826dx5fb1405ppnjjm4700000gn/T/ipykernel_4906/1546787270.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mac2=torch.tensor(torch.rand(11,100)).to(device)


In [87]:
context_length = attention_scores.shape[0]
mask = torch.tril( 
    torch.ones(context_length,context_length)
)

In [88]:
mask

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [115]:
masked_attn=attention_scores * mask

In [116]:
masked_attn

tensor([[-0.0590, -0.0000,  0.0000, -0.0000,  0.0000,  0.0000],
        [-0.0598, -0.0513,  0.0000, -0.0000,  0.0000, -0.0000],
        [-0.0502, -0.0436, -0.1549, -0.0000, -0.0000, -0.0000],
        [-0.1310, -0.0710, -0.2404, -0.1792, -0.0000, -0.0000],
        [-0.0197, -0.0217, -0.0984, -0.1233, -0.0645, -0.0000],
        [-0.0470, -0.0808,  0.0925, -0.1022,  0.0957,  0.0860]],
       grad_fn=<MulBackward0>)

In [113]:
masked_attn = masked_attn / masked_attn.sum(-1,keepdim=True)

In [118]:
masked_attn

tensor([[-0.0590, -0.0000,  0.0000, -0.0000,  0.0000,  0.0000],
        [-0.0598, -0.0513,  0.0000, -0.0000,  0.0000, -0.0000],
        [-0.0502, -0.0436, -0.1549, -0.0000, -0.0000, -0.0000],
        [-0.1310, -0.0710, -0.2404, -0.1792, -0.0000, -0.0000],
        [-0.0197, -0.0217, -0.0984, -0.1233, -0.0645, -0.0000],
        [-0.0470, -0.0808,  0.0925, -0.1022,  0.0957,  0.0860]],
       grad_fn=<MulBackward0>)

In [119]:
masked_attn.sum(-1)

tensor([-0.0590, -0.1111, -0.2487, -0.6217, -0.3276,  0.0441],
       grad_fn=<SumBackward1>)

In [133]:
mask = torch.triu(
    torch.ones(context_length,context_length)
    ,diagonal=1
)
mask


tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [134]:

masked = attention_scores.masked_fill(
    mask.bool(),
    -torch.inf
)

In [135]:
masked

tensor([[-0.0590,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.0598, -0.0513,    -inf,    -inf,    -inf,    -inf],
        [-0.0502, -0.0436, -0.1549,    -inf,    -inf,    -inf],
        [-0.1310, -0.0710, -0.2404, -0.1792,    -inf,    -inf],
        [-0.0197, -0.0217, -0.0984, -0.1233, -0.0645,    -inf],
        [-0.0470, -0.0808,  0.0925, -0.1022,  0.0957,  0.0860]],
       grad_fn=<MaskedFillBackward0>)

In [136]:
attn_weights = torch.softmax(
    masked / keys.shape[-1] ** 0.5,dim=-1
)



In [137]:
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4988, 0.5012, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3395, 0.3408, 0.3196, 0.0000, 0.0000, 0.0000],
        [0.2534, 0.2623, 0.2379, 0.2464, 0.0000, 0.0000],
        [0.2053, 0.2051, 0.1962, 0.1934, 0.2001, 0.0000],
        [0.1613, 0.1582, 0.1749, 0.1563, 0.1752, 0.1742]],
       grad_fn=<SoftmaxBackward0>)

In [139]:
# now we apply dropout to the attention scores 

torch.manual_seed(69)

dropout=torch.nn.Dropout(0.5)
example=torch.ones(6,6)
print(dropout(example))

tensor([[0., 0., 2., 0., 0., 2.],
        [0., 0., 0., 2., 0., 0.],
        [2., 0., 0., 0., 0., 0.],
        [0., 0., 0., 2., 2., 0.],
        [2., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.]])


In [140]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6791, 0.6817, 0.6392, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5246, 0.4758, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4101, 0.0000, 0.3868, 0.0000, 0.0000],
        [0.0000, 0.3164, 0.3497, 0.3125, 0.3503, 0.0000]],
       grad_fn=<MulBackward0>)


In [141]:
batch=torch.stack((inputs,inputs),dim=0)
print(batch.shape)

torch.Size([2, 6, 4])


In [180]:
class CausalAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,bias_=False):
        super().__init__()
        self.d_out=d_out
        self.w_query=torch.nn.Linear(
            d_in,d_out,bias_
        )
        self.w_key=torch.nn.Linear(
            d_in,d_out,bias_
        )
        self.w_value=nn.Linear(
            d_in,d_out,bias_
        )
        self.dropout=nn.Dropout(
            dropout
        )
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length,context_length),diagonal=1)

        )


    def forward(self,x):

        b ,num_tokens,d_in = x.shape
        keys=self.w_key(x)
        values=self.w_value(x)
        queries=self.w_query(x)

        attn_scores=queries @ keys.transpose(1,2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens , :num_tokens],-torch.inf)
        attn_weights=torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5 , dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec= attn_weights @ values

        return context_vec









In [181]:
torch.manual_seed(123)

context_length = batch.shape[1]

ca=CausalAttention(
    d_in=4,
    d_out=2,
    context_length=context_length,

    dropout=0.2


)
ca(batch)

tensor([[[ 0.0173,  0.8058],
         [-0.0473,  0.6571],
         [-0.0305,  0.4268],
         [-0.2195,  0.6064],
         [-0.1743,  0.4883],
         [-0.0358,  0.4079]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0057,  0.2674],
         [-0.2195,  0.6064],
         [-0.1925,  0.5513],
         [-0.0865,  0.5450]]], grad_fn=<UnsafeViewBackward0>)

In [201]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in , d_out , num_heads, dropout , context_length, bias_=False ):
        super().__init__()
        self.head_dim = d_out / num_heads

        self.heads = nn.ModuleList([
            CausalAttention(
                d_in, d_out,
                context_length, dropout, bias_
            )
            for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat(
            [head(x) for head in self.heads], dim=-1
        )

In [202]:
torch.manual_seed(69)
context_length = batch.shape[1]
print(batch.shape[1])
print(batch.shape[2])

mha=MultiHeadAttention(
    d_in=4,
  
    dropout=0.2,
    context_length=context_length,
    num_heads=4,
    d_out=4
    

)
context_vectors=mha(batch)


6
4


In [203]:
context_vectors.shape

torch.Size([2, 6, 16])

In [236]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
        "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                      diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        
        # Linear transformations
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        # Reshape to include heads dimension
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Compute attention scores
        attn_scores = queries @ keys.transpose(-2, -1)
        
        # Apply mask
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        # Compute attention weights
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # Compute context vectors
        context_vec = attn_weights @ values
        context_vec = context_vec.transpose(1, 2).contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)
        
        return context_vec

In [239]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in=d_in, d_out=d_out, context_length=context_length, dropout=0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[ 0.7047, -0.3305],
         [ 0.6953, -0.3733],
         [ 0.7038, -0.3727],
         [ 0.7317, -0.3482],
         [ 0.7134, -0.3794],
         [ 0.7132, -0.3585]],

        [[ 0.7047, -0.3305],
         [ 0.6953, -0.3733],
         [ 0.7038, -0.3727],
         [ 0.7317, -0.3482],
         [ 0.7134, -0.3794],
         [ 0.7132, -0.3585]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [241]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573], [0.8993, 0.0390, 0.9268, 0.7388],
[0.7179, 0.7058, 0.9156, 0.4340]],

[[0.0772, 0.3565, 0.1479, 0.5331],
[0.4066, 0.2318, 0.4545, 0.9737],
[0.4606, 0.5159, 0.4220, 0.5786]]]])
a

tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
          [0.8993, 0.0390, 0.9268, 0.7388],
          [0.7179, 0.7058, 0.9156, 0.4340]],

         [[0.0772, 0.3565, 0.1479, 0.5331],
          [0.4066, 0.2318, 0.4545, 0.9737],
          [0.4606, 0.5159, 0.4220, 0.5786]]]])

In [243]:
a.shape

torch.Size([1, 2, 3, 4])

In [246]:
multiHeadAttentionGPT2=MultiHeadAttention(
    d_in=768,d_out=768,dropout=0.2,context_length=1024,num_heads=12,
)